In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
tf.__version__

'2.4.0'

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
batch_size = 128

In [5]:
x_train = x_train / 255
x_test = x_test / 255

In [6]:
n_train_batch = x_train.shape[0] // batch_size

In [7]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1], name='x')
y = tf.placeholder(tf.float32, [None, 10], name='y')

In [8]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [9]:
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

In [10]:
conv2d_0_w = tf.get_variable('conv2d_0', [3, 3, 1, 32])
conv2d_1_w = tf.get_variable('conv2d_1', [3, 3, 32, 64])
fc_0_w = tf.get_variable('fc_0_w', [5*5*64, 1024])
fc_0_b = tf.get_variable('fc_0_b', [1024])
fc_1_w = tf.get_variable('fc_1_w', [1024, 10])
fc_1_b = tf.get_variable('fc_1_b', [10])

In [11]:
o0 = tf.nn.conv2d(x, conv2d_0_w, strides=[1, 1, 1, 1], padding='VALID')
o1 = tf.nn.relu(o0)
o2 = tf.nn.max_pool2d(o1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

In [12]:
o3 = tf.nn.conv2d(o2, conv2d_1_w, strides=[1, 1, 1, 1], padding='VALID')
o4 = tf.nn.relu(o3)
o5 = tf.nn.max_pool2d(o4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

In [13]:
o6 = tf.reshape(o5, (-1, 5*5*64))

In [14]:
o7 = tf.nn.relu(tf.add(tf.matmul(o6, fc_0_w), fc_0_b))

In [15]:
o5

<tf.Tensor 'MaxPool2d_1:0' shape=(?, 5, 5, 64) dtype=float32>

In [16]:
prediction =tf.add(tf.matmul(o7, fc_1_w), fc_1_b)

In [17]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [18]:
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

In [19]:
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [20]:
init = tf.global_variables_initializer()

In [21]:
epoch = 5

In [22]:
with tf.Session() as sess:
    sess.run(init)
    for e in range(epoch):
        for batch in range(n_train_batch):
            start_index = batch * batch_size
            end_index = (batch + 1) * batch_size

            batch_x, batch_y = x_train[start_index:end_index], y_train[start_index:end_index]
        
            sess.run(train_step, {x : batch_x, y : batch_y})
        acc, l = sess.run([accuracy, loss], {x : x_test, y : y_test})
        print("Iter: " + str(e + 1) + " Accuracy: " + str(acc) + " Loss: "+ str(l) )

Iter: 1 Accuracy: 0.9805 Loss: 0.057715748
Iter: 2 Accuracy: 0.9869 Loss: 0.03685489
Iter: 3 Accuracy: 0.9885 Loss: 0.033044815
Iter: 4 Accuracy: 0.9879 Loss: 0.03593344
Iter: 5 Accuracy: 0.9863 Loss: 0.047093477
